<a href="https://colab.research.google.com/github/Blova-TB/OptimisationUQAC/blob/Probleme-4/template/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMPL Model Colaboratory Template

Description: Basic notebook template for the AMPL Colab repository

Tags: amplpy, template, highlights

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

In [15]:
# Install dependencies
%pip install -q amplpy pandas

In [16]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


## Use `%%ampl_eval` to evaluate AMPL commands and declarations

In [17]:
%%ampl_eval

set POUDRE; # A ou B
set PRODUIT; # enceinte ou allaite
set NUTRIMENT; # Lipide ou Proteine
set PRODUCTEUR; # <-- Modif

param prix {PRODUIT} >= 0;
param cout {POUDRE,PRODUCTEUR} >= 0; # <-- Modif
param maxVenteProduit {PRODUIT} >= 0;
param minVenteProduit {PRODUIT} >= 0;
param disponibilitePoudre {POUDRE,PRODUCTEUR} >= 0; # <-- Modif
param compositionMaxProduit {PRODUIT,NUTRIMENT} >= 0;
param compositionMinProduit {PRODUIT,NUTRIMENT} >= 0;
param compositionPoudre {POUDRE,NUTRIMENT} >= 0;

var achat {POUDRE,PRODUIT,PRODUCTEUR} >=0; # <-- Modif
var vente {prod in PRODUIT} >= minVenteProduit[prod], <= maxVenteProduit[prod];

maximize Total_Cost: sum {prod in PRODUIT} vente[prod] * prix[prod] - sum {po in POUDRE, prod in PRODUIT, pdct in PRODUCTEUR} achat[po,prod,pdct] * cout[po,pdct]; # <-- Modif

subject to Max_Composition_Nutriment {prod in PRODUIT, nut in NUTRIMENT}:
    vente[prod] * compositionMaxProduit[prod,nut] >= sum {po in POUDRE, pdct in PRODUCTEUR} achat[po,prod,pdct] * compositionPoudre[po,nut]; # <-- modif

subject to Min_Composition_Nutriment {prod in PRODUIT, nut in NUTRIMENT}:
    vente[prod] * compositionMinProduit[prod,nut] <= sum {po in POUDRE, pdct in PRODUCTEUR} achat[po,prod,pdct] * compositionPoudre[po,nut]; # <-- modif

subject to Disponibilite_Poudre {po in POUDRE, pdct in PRODUCTEUR}:
    sum {prod in PRODUIT} achat[po,prod,pdct] <= disponibilitePoudre[po,pdct]; # <-- modif

subject to six_foix_plus_chez_A:
    sum {prod in PRODUIT} achat['A', prod, 'ProdA'] >= 6 * sum {prod in PRODUIT} achat['A', prod, 'ProdB'];

## Load data directly from Python data structures using [amplpy](https://amplpy.readthedocs.io/)

In [18]:
poudre = ["A","B"]
ampl.set["POUDRE"] = poudre

produit = ["enceinte","allaite"]
ampl.set["PRODUIT"] = produit

nut = ["Lipide","Proteine"]
ampl.set["NUTRIMENT"] = nut

pdct = ["ProdA","ProdB"]
ampl.set["PRODUCTEUR"] = pdct

ampl.param["prix"] = {
    "enceinte": 18,
    "allaite": 25
}

ampl.param["cout"] = {
    ("A", "ProdA"): 22,
    ("A", "ProdB"): 16,
    ("B", "ProdA"): 15,
    ("B", "ProdB"): 0
}

ampl.param["maxVenteProduit"] = {
    "enceinte": 20,
    "allaite": 30
}

ampl.param["minVenteProduit"] = {
    "enceinte": 15,
    "allaite": 22
}

ampl.param["disponibilitePoudre"] = {
    ("A", "ProdA"): 18,
    ("A", "ProdB"): 9,
    ("B", "ProdA"): 31,
    ("B", "ProdB"): 0
}

ampl.param["compositionMaxProduit"] = {
    ("enceinte","Proteine"): 20,
    ("enceinte","Lipide"): 40,
    ("allaite","Proteine"):20,
    ("allaite","Lipide"):45
}

ampl.param["compositionMinProduit"] = {
    ("enceinte","Proteine"): 12,
    ("enceinte","Lipide"): 30,
    ("allaite","Proteine"):12,
    ("allaite","Lipide"):20
}

ampl.param["compositionPoudre"] = {
    ("A","Proteine"):35,
    ("A","Lipide"):38,
    ("B","Proteine"):5,
    ("B","Lipide"):32
}

## Solve with HiGHS

In [19]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"
ampl.solve()

HiGHS 1.11.0:   tech:outlev = 1

AMPL MP initial flat model has 8 variables (0 integer, 0 binary);
Objectives: 1 linear; 
Constraints:  12 linear;

AMPL MP did not modify the model.


Running HiGHS 1.11.0 (git hash: 364c83a51): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 12 rows; 8 cols; 42 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+01]
  Cost   [2e+01, 2e+01]
  Bound  [2e+01, 3e+01]
  RHS    [9e+00, 3e+01]
Presolving model
12 rows, 8 cols, 42 nonzeros  0s
12 rows, 8 cols, 42 nonzeros  0s
Presolve : Reductions: rows 12(-0); columns 8(-0); elements 42(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          6     5.0023041475e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  5.0023041475e+02
P-D objective error :  5.6760501446e-17
HiGHS run time      :          0.00

## Retrieve solution as a pandas dataframe

In [22]:
ampl.var["achat"].to_pandas()

achat.val
index0 index1   index2           
A      allaite  ProdA    7.133641
                ProdB    2.027650
       enceinte ProdA    5.032258
                ProdB    0.000000
B      allaite  ProdA    7.870968
                ProdB    0.000000
       enceinte ProdA   12.774194
                ProdB    0.000000

In [21]:
ampl.var["vente"].to_pandas()

,vente.val
allaite,30
enceinte,20
